In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1691410809424_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('trusted_bi_politica_credito_imob_modelo_001').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_AULA_ = '20230804'
_REF_ = '202001'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
raw ='t_bi_politica_credito_imob_modelo_001'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
bucket_trusted = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/003_trusted'.format(dia=_DIA_AULA_)
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_AULA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_AULA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
output_trusted = 'trusted_bi_politica_credito_imob_modelo_001'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
full_path_raw = 's3://{bkt}/{file}'.format(bkt=bucket_raw,file=raw)
print(full_path_raw)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/002_raw/t_bi_politica_credito_imob_modelo_001

In [10]:
path_lake = os.path.join(full_path_raw)
raw_00=sqlContext.read.format("parquet").load(path_lake)
raw_00.registerTempTable("raw_00")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
raw_00.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------------+--------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+-------------+----------------------------+-----------------+
|ref   |ts_file_generation|ts_proc       |cd_codigo|dc_regiao   |sexo|fl_application|RFM     |dc_est_civ|dc_tp_cli |nr_score|nr_idade|vl_renda|vl_tomado|fl_restritivo|fl_atraso|fl_app|UF_RG|UF_Loja|decision_1|decision_2|ref_partition|ts_file_generation_partition|ts_proc_partition|
+------+------------------+--------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+-------------+----------------------------+-----------------+
|202308|100               |20230806153820|3        |Centro-Oeste|M   |0             |Em Risco|Solteiro  |Já cliente|128     |34      |3180    |612      |1

In [12]:
# +------+------------------+--------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+-------------+----------------------------+-----------------+
# |ref   |ts_file_generation|ts_proc       |cd_codigo|dc_regiao   |sexo|fl_application|RFM     |dc_est_civ|dc_tp_cli |nr_score|nr_idade|vl_renda|vl_tomado|fl_restritivo|fl_atraso|fl_app|UF_RG|UF_Loja|decision_1|decision_2|ref_partition|ts_file_generation_partition|ts_proc_partition|
# +------+------------------+--------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+-------------+----------------------------+-----------------+
# |202308|100               |20230806153820|3        |Centro-Oeste|M   |0             |Em Risco|Solteiro  |Já cliente|128     |34      |3180    |612      |1            |0        |0     |4    |RJ     |Negado    |0         |202308       |100                         |20230806153820   |
# |202308|100               |20230806153820|4        |Sudeste     |M   |0             |Em Risco|Solteiro  |Já cliente|377     |49      |2894    |919      |0            |1        |1     |9    |MT     |Negado    |0         |202308       |100                         |20230806153820   |
# |202308|100               |20230806153820|5        |Norte       |F   |0             |Em Risco|Divorciado|Novo      |911     |69      |2319    |1426     |0            |1        |0     |1    |PI     |Negado    |0         |202308       |100                         |20230806153820   |
# |202308|100               |20230806153820|9        |Norte       |M   |0             |Em Risco|Casado    |Já cliente|350     |35      |3744    |1807     |1            |0        |0     |5    |RJ     |Negado    |0         |202308       |100                         |20230806153820   |
# |202308|100               |20230806153820|10       |Sul         |M   |0             |Em Risco|Viuvo     |Já cliente|852     |68      |1560    |2669     |0            |0        |0     |9    |BA     |Negado    |0         |202308       |100                         |20230806153820   |
# +------+------------------+--------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+-------------+----------------------------+-----------------+
# only showing top 5 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
print('lista de colunas para tipar')
for col in raw_00.columns:
    print('cast(' + col + ' as) as ' + col + ',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lista de colunas para tipar
cast(ref as) as ref,
cast(ts_file_generation as) as ts_file_generation,
cast(ts_proc as) as ts_proc,
cast(cd_codigo as) as cd_codigo,
cast(dc_regiao as) as dc_regiao,
cast(sexo as) as sexo,
cast(fl_application as) as fl_application,
cast(RFM as) as RFM,
cast(dc_est_civ as) as dc_est_civ,
cast(dc_tp_cli as) as dc_tp_cli,
cast(nr_score as) as nr_score,
cast(nr_idade as) as nr_idade,
cast(vl_renda as) as vl_renda,
cast(vl_tomado as) as vl_tomado,
cast(fl_restritivo as) as fl_restritivo,
cast(fl_atraso as) as fl_atraso,
cast(fl_app as) as fl_app,
cast(UF_RG as) as UF_RG,
cast(UF_Loja as) as UF_Loja,
cast(decision_1 as) as decision_1,
cast(decision_2 as) as decision_2,
cast(ref_partition as) as ref_partition,
cast(ts_file_generation_partition as) as ts_file_generation_partition,
cast(ts_proc_partition as) as ts_proc_partition,

In [33]:
lake = spark.sql(     
    """
        select
            -- padrao para todas as cargas no datalake--
            
            ref,
            ref_partition,
            
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
            -- campos do arquivo --

            cast(cd_codigo as int) as cd_codigo,
            cast(dc_regiao as string) as dc_regiao,
            cast(sexo as string) as sexo,
            cast(fl_application as int) as fl_application,
            cast(RFM as string) as RFM,
            cast(dc_est_civ as string) as dc_est_civ,
            cast(dc_tp_cli as string) as dc_tp_cli,
            cast(nr_score as int) as nr_score,
            cast(nr_idade as int) as nr_idade,
            cast(vl_renda as int) as vl_renda,
            cast(vl_tomado as int) as vl_tomado,
            cast(fl_restritivo as int) as fl_restritivo,
            cast(fl_atraso as int) as fl_atraso,
            cast(fl_app as int) as fl_app,
            cast(UF_RG as int) as UF_RG,
            cast(UF_Loja as string) as UF_Loja,
            cast(decision_1 as string) as decision_1,
            cast(decision_2 as int) as decision_2

            
        from
            raw_00
        --where
            --ref_partition = {_ref_}
        --    ref = {_ref_}
            
            
    """.format(pdthproc=dthproc,_ref_ = _REF_))
lake.registerTempTable("lake")
lake.cache()
lake.count()  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5000

In [34]:
lake.explain()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
InMemoryTableScan [ref#0, ref_partition#21, ts_proc#2118L, ts_proc_partition#2119L, cd_codigo#2120, dc_regiao#2121, sexo#2122, fl_application#2123, RFM#2124, dc_est_civ#2125, dc_tp_cli#2126, nr_score#2127, nr_idade#2128, vl_renda#2129, vl_tomado#2130, fl_restritivo#2131, fl_atraso#2132, fl_app#2133, UF_RG#2134, UF_Loja#2135, decision_1#2136, decision_2#2137]
   +- InMemoryRelation [ref#0, ref_partition#21, ts_proc#2118L, ts_proc_partition#2119L, cd_codigo#2120, dc_regiao#2121, sexo#2122, fl_application#2123, RFM#2124, dc_est_civ#2125, dc_tp_cli#2126, nr_score#2127, nr_idade#2128, vl_renda#2129, vl_tomado#2130, fl_restritivo#2131, fl_atraso#2132, fl_app#2133, UF_RG#2134, UF_Loja#2135, decision_1#2136, decision_2#2137], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(1) Project [ref#0, ref_partition#21, 20230807092900 AS ts_proc#2118L, 20230807092900 AS ts_proc_partition#2119L, cast(cd_codigo#3 as int) AS cd_codigo#2120, dc_regiao#4, sexo#5, cast(fl

In [35]:
lake.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_proc: long (nullable = false)
 |-- ts_proc_partition: long (nullable = false)
 |-- cd_codigo: integer (nullable = true)
 |-- dc_regiao: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- fl_application: integer (nullable = true)
 |-- RFM: string (nullable = true)
 |-- dc_est_civ: string (nullable = true)
 |-- dc_tp_cli: string (nullable = true)
 |-- nr_score: integer (nullable = true)
 |-- nr_idade: integer (nullable = true)
 |-- vl_renda: integer (nullable = true)
 |-- vl_tomado: integer (nullable = true)
 |-- fl_restritivo: integer (nullable = true)
 |-- fl_atraso: integer (nullable = true)
 |-- fl_app: integer (nullable = true)
 |-- UF_RG: integer (nullable = true)
 |-- UF_Loja: string (nullable = true)
 |-- decision_1: string (nullable = true)
 |-- decision_2: integer (nullable = true)

In [36]:
lake.show(4,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+--------------+-----------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+
|ref   |ref_partition|ts_proc       |ts_proc_partition|cd_codigo|dc_regiao   |sexo|fl_application|RFM     |dc_est_civ|dc_tp_cli |nr_score|nr_idade|vl_renda|vl_tomado|fl_restritivo|fl_atraso|fl_app|UF_RG|UF_Loja|decision_1|decision_2|
+------+-------------+--------------+-----------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+
|202308|202308       |20230807092900|20230807092900   |3        |Centro-Oeste|M   |0             |Em Risco|Solteiro  |Já cliente|128     |34      |3180    |612      |1            |0        |0     |4    |RJ     |Negado    |0         |
|202308|202308       |20230807092900|20230807092900   |4        

In [37]:
# +------+-------------+--------------+-----------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+
# |ref   |ref_partition|ts_proc       |ts_proc_partition|cd_codigo|dc_regiao   |sexo|fl_application|RFM     |dc_est_civ|dc_tp_cli |nr_score|nr_idade|vl_renda|vl_tomado|fl_restritivo|fl_atraso|fl_app|UF_RG|UF_Loja|decision_1|decision_2|
# +------+-------------+--------------+-----------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+
# |202308|202308       |20230807092900|20230807092900   |3        |Centro-Oeste|M   |0             |Em Risco|Solteiro  |Já cliente|128     |34      |3180    |612      |1            |0        |0     |4    |RJ     |Negado    |0         |
# |202308|202308       |20230807092900|20230807092900   |4        |Sudeste     |M   |0             |Em Risco|Solteiro  |Já cliente|377     |49      |2894    |919      |0            |1        |1     |9    |MT     |Negado    |0         |
# |202308|202308       |20230807092900|20230807092900   |5        |Norte       |F   |0             |Em Risco|Divorciado|Novo      |911     |69      |2319    |1426     |0            |1        |0     |1    |PI     |Negado    |0         |
# |202308|202308       |20230807092900|20230807092900   |9        |Norte       |M   |0             |Em Risco|Casado    |Já cliente|350     |35      |3744    |1807     |1            |0        |0     |5    |RJ     |Negado    |0         |
# +------+-------------+--------------+-----------------+---------+------------+----+--------------+--------+----------+----------+--------+--------+--------+---------+-------------+---------+------+-----+-------+----------+----------+
# only showing top 4 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
path_trusted = os.path.join('s3://',bucket_trusted,output_trusted)
print(path_trusted)

lake.coalesce(1).write. \
partitionBy("ref_partition","ts_proc_partition"). \
parquet(path_trusted,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/003_trusted/trusted_bi_politica_credito_imob_modelo_001

In [39]:
varDataFile = output_trusted

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
controle = spark.sql(    
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,
                
                ref,
                ref_partition,
                
                ts_proc,
                ts_proc_partition,
                
                count(*) as qtd_registros
            from 
                lake as a
            group by
                1,2,3,4,5,6
            order by
                1,2,3,4,5,6
        """.format(tb=varDataFile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
controle.registerTempTable("controle")
controle.cache()
qtd=controle.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
controle.show(qtd,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------+-------------------------------------------+------+-------------+--------------+-----------------+-------------+
|name_file                                  |name_file_partition                        |ref   |ref_partition|ts_proc       |ts_proc_partition|qtd_registros|
+-------------------------------------------+-------------------------------------------+------+-------------+--------------+-----------------+-------------+
|trusted_bi_politica_credito_imob_modelo_001|trusted_bi_politica_credito_imob_modelo_001|202308|202308       |20230807092900|20230807092900   |5000         |
+-------------------------------------------+-------------------------------------------+------+-------------+--------------+-----------------+-------------+

In [43]:
# +-------------------------------------------+-------------------------------------------+------+-------------+--------------+-----------------+-------------+
# |name_file                                  |name_file_partition                        |ref   |ref_partition|ts_proc       |ts_proc_partition|qtd_registros|
# +-------------------------------------------+-------------------------------------------+------+-------------+--------------+-----------------+-------------+
# |trusted_bi_politica_credito_imob_modelo_001|trusted_bi_politica_credito_imob_modelo_001|202308|202308       |20230807092900|20230807092900   |5000         |
# +-------------------------------------------+-------------------------------------------+------+-------------+--------------+-----------------+-------------+

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
path_control = os.path.join('s3://',bucket_control,'tb_0002_controle_procesamento_trusted')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/005_control/tb_0002_controle_procesamento_trusted